In [3]:
import pandas as pd
import numpy as np
from faker import Faker

In [4]:
# Initialize Faker
fake = Faker()

In [6]:
print("Loading data...")
df_raw = pd.read_excel("C:/Users/eyobh/Downloads/online+retail+ii/online_retail_II.xlsx")

Loading data...


In [ ]:
df_clean = df_raw.dropna(subset=['Customer ID'])
df_clean = df_clean[df_clean['Quantity'] > 0] 
df_clean['Customer ID'] = df_clean['Customer ID'].astype(int)

In [8]:
unique_customers = df_clean['Customer ID'].unique()
print(f"Found {len(unique_customers)} unique customers.")

Found 4314 unique customers.


In [9]:
print("Generating synthetic customer profiles...")
customer_data = []

for cust_id in unique_customers:
    customer_data.append({
        'Customer_ID': cust_id,
        'Name': fake.name(),
        'Gender': np.random.choice(['M', 'F'], p=[0.45, 0.55]),
        'Age': np.random.randint(18, 70),
        'Country': 'United Kingdom', # Keeping it simple or mapping from original data
        'Email': fake.email()
    })

df_customers = pd.DataFrame(customer_data)

print("Data Preparation Complete.")
print(df_customers.head())

Generating synthetic customer profiles...
Data Preparation Complete.
   Customer_ID               Name Gender  Age         Country  \
0        13085  Stephen Hernandez      M   60  United Kingdom   
1        13078         Steven Lin      F   34  United Kingdom   
2        15362     Jason Martinez      F   43  United Kingdom   
3        18102     Jennifer Mejia      M   45  United Kingdom   
4        12682      Joseph Kelley      M   30  United Kingdom   

                        Email  
0       kenneth63@example.org  
1  rileychristina@example.com  
2          hsmith@example.net  
3      ealexander@example.com  
4       barbara63@example.com  


In [10]:
# 1. Create Dim_Product (Normalization)
df_products = df_clean[['StockCode', 'Description', 'Price']].drop_duplicates(subset='StockCode')
df_products.rename(columns={'Price': 'Unit_Cost'}, inplace=True) # Renaming for clarity in DB
print(f"Products extracted: {len(df_products)}")

Products extracted: 4017


In [11]:
# 2. RFM Analysis (The Marketing Analytics Engine)
# Calculate Recency, Frequency, and Monetary value for each customer
snapshot_date = df_clean['InvoiceDate'].max() + pd.DateOffset(days=1)

rfm = df_clean.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, # Recency
    'Invoice': 'nunique',                                    # Frequency
    'Price': 'sum'                                           # Monetary (Approximation)
}).rename(columns={'InvoiceDate': 'Recency', 'Invoice': 'Frequency', 'Price': 'Monetary'})

In [12]:
# Create Scores (1-4 scale using quartiles)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1]) # Low recency is good (4)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

# Create Segments
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

def segment_customer(score):
    if score[0] == '4' and score[1] == '4': return 'Champions'
    if score[0] == '4' and score[1] == '3': return 'Loyal'
    if score[0] == '3' and score[1] == '3': return 'Potential Loyalist'
    if score[0] == '1' and score[1] == '1': return 'Lost'
    return 'Standard'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)

In [13]:
# 3. Merge RFM into Customer Table
# We merge the new marketing data into our demographic table
df_customers_final = df_customers.merge(rfm[['Segment', 'Recency', 'Monetary']], left_on='Customer_ID', right_index=True, how='left')
df_customers_final['Segment'].fillna('New Customer', inplace=True)

print("RFM Segmentation Complete. Examples:")
print(df_customers_final[['Customer_ID', 'Segment']].head())

RFM Segmentation Complete. Examples:
   Customer_ID    Segment
0        13085   Standard
1        13078  Champions
2        15362   Standard
3        18102  Champions
4        12682  Champions


C:\Users\eyobh\AppData\Local\Temp\ipykernel_21248\3419860148.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customers_final['Segment'].fillna('New Customer', inplace=True)


In [20]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

user = 'root'
password = 'Miki@!23456' 
host = 'localhost'
db_name = 'retail_analytics_db'
port = "3306"

# 1. Setup & Database Creation
encoded_password = quote_plus(password)

# Connect to Server Root first
root_connection_str = f"mysql+pymysql://{user}:{encoded_password}@{host}:{port}?charset=utf8mb4"
root_engine = create_engine(root_connection_str, pool_pre_ping=True)

# Create the database
with root_engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {db_name} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
    print(f"✅ Database {db_name} created (or verified).")

# ---------------------------------------------------------
# 2. CRITICAL STEP: Connect to the specific Database now
# ---------------------------------------------------------
db_connection_str = f"mysql+pymysql://{user}:{encoded_password}@{host}:{port}/{db_name}?charset=utf8mb4"
db_engine = create_engine(db_connection_str, pool_pre_ping=True)

# 3. Load DataFrames (Using db_engine, not root_engine)
print("Uploading to MySQL...")

try:
    # Dimension Tables
    df_customers_final.to_sql('dim_customer', con=db_engine, if_exists='replace', index=False)
    print("✓ Dim_Customer Loaded")
    
    df_products.to_sql('dim_product', con=db_engine, if_exists='replace', index=False)
    print("✓ Dim_Product Loaded")
    
    # Fact Table
    df_sales_fact = df_clean[['Invoice', 'StockCode', 'Customer ID', 'Quantity', 'Price', 'InvoiceDate']]
    df_sales_fact.to_sql('fact_sales', con=db_engine, if_exists='replace', index=False)
    print("✓ Fact_Sales Loaded")

    print("SUCCESS: Data Warehouse successfully populated.")

except Exception as e:
    print(f"Error: {e}")

✅ Database retail_analytics_db created (or verified).
Uploading to MySQL...
✓ Dim_Customer Loaded
✓ Dim_Product Loaded
✓ Fact_Sales Loaded
SUCCESS: Data Warehouse successfully populated.
